FAISS Set Up

In [1]:
!pip install numpy
!pip install sentence-transformers
!pip install openai
!pip install faiss-cpu
!pip install transformers


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 10.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import json  # Built-in, no installation needed
import numpy as np
from sentence_transformers import SentenceTransformer
import openai
import faiss  # For building and querying the vector database
from transformers import pipeline
import os
import openai
import getpass

In [5]:

from sentence_transformers import SentenceTransformer
import json

# Load the JSON file
with open("metdata.json", "r") as file:
    data = json.load(file)

# Extract paintings data
paintings = data["paintings"]

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # A lightweight model for embeddings

# Generate embeddings for each painting
texts = [
    f"{painting['title']} {painting['description']} {' '.join(painting.get('tags', []))}"
    for painting in paintings
]

# Encode the texts to create embeddings
embeddings = model.encode(texts, convert_to_tensor=False)  # Returns a NumPy array

# Display the first embedding for verification
print(f"First embedding for '{paintings[0]['title']}':", embeddings[0])

First embedding for 'Joan of Arc': [ 1.28659224e-02  7.52580911e-02  2.63640005e-02 -6.11151941e-02
  5.59599102e-02  1.04252726e-01 -4.39553373e-02  6.11743852e-02
  2.49663983e-02 -5.65736294e-02  2.34689768e-02 -6.01532869e-02
  5.46257198e-02 -3.85121144e-02 -2.49282140e-02 -8.73401947e-03
 -4.41298038e-02  1.16389252e-01 -4.19332273e-02  8.00186694e-02
 -5.08325808e-02 -6.55834153e-02  2.17069723e-02  2.89588571e-02
  8.65611434e-02 -3.24539393e-02  3.39159034e-02 -4.38671969e-02
 -5.75492978e-02  6.13340400e-02 -4.35420731e-03  1.16772009e-02
 -8.80586207e-02  2.52330117e-02 -6.72517270e-02 -6.30604699e-02
  1.71815429e-03  2.78348252e-02 -4.09635939e-02  6.47060499e-02
 -1.01237483e-01 -4.32756990e-02 -8.72333422e-02 -7.28660375e-02
  5.87235838e-02 -4.21024188e-02  2.00012848e-02  3.00585404e-02
  1.56338345e-02  1.17665008e-02  7.85478055e-02  4.31159809e-02
 -2.50381976e-03 -4.93614487e-02 -7.88974911e-02 -3.49021666e-02
  4.77681048e-02  1.61563866e-02  4.05374914e-02  6.142

In [6]:
import faiss
import numpy as np

# Convert embeddings to a NumPy array
embedding_array = np.array(embeddings).astype('float32')

# Initialize FAISS index
dimension = embedding_array.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance (Euclidean)
index.add(embedding_array)  # Add embeddings to the index

In [7]:
faiss.write_index(index, "paintings_index.faiss")


In [25]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your GPT API key: ")

In [26]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [27]:
# Load the FAISS index
index = faiss.read_index("paintings_index.faiss")

# Load the JSON metadata
with open("metdata.json", "r") as file:
    data = json.load(file)

paintings = data["paintings"]  # List of painting metadata

In [28]:
# Search function
def search_paintings(query, top_k=3):
    """
    Searches the FAISS index for the most relevant paintings based on a user query.

    Args:
        query (str): The user's input query.
        top_k (int): Number of top results to return.
    
    Returns:
        list: List of dictionaries containing painting metadata.
    """
    query_embedding = model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, top_k)  # FAISS search
    results = [paintings[i] for i in indices[0]]  # Retrieve painting metadata
    return results

In [31]:
client = openai.OpenAI(
    api_key = os.getenv("OPENAI_API_KEY")
)

In [46]:
# Generate response function
def generate_response(query):
    """
    Generates a conversational response based on the user's query and related paintings.

    Args:
        query (str): The user's input query.
    
    Returns:
        str: The generated conversational response.
    """
    # Search for relevant paintings
    results = search_paintings(query)

    # Construct a conversational prompt
    painting_info = "\n".join(
        [f"Title: {p['title']}\nDescription: {p['description']}" for p in results]
    )
    print(painting_info)
    prompt = f"""
    User asked: {query}
    Here is some information about related paintings:
    {painting_info}
    Please respond with the paintings that are listed above. 
    Respond to the user in a conversational tone and provide a brief history of the painting. Then ask the user if they have any questions about this painting.
    """
    
    # Generate response from the OpenAI API
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
            {"role": "system", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Test the response generation
query = "What painting is the one with three girls in red dresses peeling potatoes?"
response = generate_response(query) 
print(response)

Title: The Three Sisters
Description: In the 1890s Frederic’s paintings of impoverished workers and peasants in his native Belgium were celebrated for their forthrightness and arresting intensity. Here, the humdrum activity of peeling potatoes is vivified by the girls’ bright red dresses and gleaming red-gold and blond hair. Their downcast eyes and serene expressions recall representations of the young Virgin Mary in sixteenth-century Flemish art, which Frederic greatly admired. Nothing is known of the sitters beyond the painting’s title, which identifies them as sisters; the two eldest are so uncannily alike that they appear to be twins.
Title: La Coiffure
Description: This painting shows a woman arranging another woman's hair and a child in the foreground, with simplified forms and earthy colors that convey a sense of timelessness. Picasso’s focus on the figure’s solid, sculptural quality reflects his interest in the classical and primitive influences during this period.
Title: The G

In [47]:
#do another test
query = "What is the Klimt painting with a woman in white dress?"
response = generate_response(query) 
print(response)

Title: Mada Primavesi
Description: Mäda Primavesi’s expression and posture convey a remarkable degree of confidence for a nine-year-old girl, even one who was, by her own account, willful and a tomboy. Klimt made numerous preliminary sketches for this portrait, experimenting with different poses, outfits, and backgrounds before deciding to show Mäda standing tall in a specially-made dress amid a profusion of springlike patterns. The picture testifies to the sophisticated taste of her parents, banker and industrialist Otto Primavesi and his wife Eugenia, who were ardent supporters of progressive Viennese art and design. In fact, Klimt soon painted Eugenia’s portrait (Toyota Municipal Museum of Art, Japan).
Title: Woman with Mimosa
Description: In Woman with Mimosa by Pierre Bonnard, a woman sits at a table with a vase of mimosa flowers, their bright yellow blossoms adding a vivid focal point to the composition. The painting captures a quiet, intimate moment as she gazes thoughtfully at 

NameError: name 'painting_info' is not defined